In [228]:
import numpy as np
from IPython.display import Audio

<center><h1>Code DTMF</h1></center>

# Les fonctions de la méthode mathématique (Utilisable en tout lieux)

In [229]:
# Norme (produit scalaire d'un vecteur avec lui même)
def norm(u):
    return np.sqrt(scalarProduct(u,u))

# Coordonées de la projection d'un vecteur v sur une famille orthonormée u
# Retourne une liste de scalaire de même longueur que la taille de u
def coordonnees(u,v):
    c = []
    for i in range(len(u)):
        c.append(scalarProduct(u[i],v))
    return c

# Projection orthogonale de v sur la famille u, qui utilise les coordonnées calculées précédemment
# Et qui renvoie c et la projection orthogonale
def projectionOrt(u,v):
    c = coordonnees(u,v)
    p = np.zeros_like(v)
    for i in range(len(u)):
        p += c[i]*u[i]
    return [c,p]


# Les fonctions propre au contexte du CodeDTMF (espace vectoriel, famille de référence et produit scalaire spécifiques)

In [230]:
# Integrale d'un signal discrétisé
def integrale(w):
    dt = (1/(len(w)-1))
    sum_w = 0
    for i in range(len(w)-1):
        sum_w += w[i]

    return sum_w*dt

# Produit scalaire 
def scalarProduct(u,v):
    return 2*integrale(np.multiply(u,v))

# Les différentes fréquences des signaux DTMF
w=[697,770,852,941,1209,1336,1477]

# fréquence d'échantillonnage, Hz, un entier
fs = 44100       

# Les sinusoïdes associées 
def function_u(i,t):
    return np.sin(t*w[(i)%7]*2*np.pi+(i//7)*np.pi/2)

# Les échantillonages correspondants (notre famille sur laquelle on va projeter)
def createOrthonormalBasis(sample_rate,signal_freq):
    u = []
    t_ech=np.linspace(0.0,1.0,fs) # intervalle d'échantillonnage
    for i in range(14):
        u.append(np.array(function_u(i,t_ech)))
    return u

# Récupération de l'enregistrement d'un son lié à un code DTMF afin de l'analyser
def getCodeFromFile(filename):
    file = open(filename, "rb")
    code_recover=np.load(file)
    file.close()
    return code_recover

# On créer un vecteur dans notre base
def codeSynthesizer(c,u):
    w = c[0] * u[0]
    for i in range(0,14):
        w += c[i] * u[i]
    return w

#Permet de déterminer l'amplitude de chaque fréquence dans l'enregistrement (même si déphaser)
def findFreqCoeff(c): #revoir plus tard pour le nom
    #Return a new array with the same shape and type as u[].
    cNew = []
    for i in range(7):
        cNew.append(c[i]**2+c[i+7]**2)
    return cNew

# Retrouve la fréquence des 2 sinusoïdes utilisées pour créer le son
def findingMainFrequencies(c):
    c_processing = c.copy()
    m1 = max(c_processing, key=abs)
    i1 = c.index(m1)

    #remove the largest number using its index
    c_processing.pop(i1)

    m2 = max(c_processing, key=abs)
    i2 = c.index(m2)
    return [i1,i2,m1,m2]

# Trouve le chiffre associé au son
def whatNumIsIt(i1, i2):
    smallest = min([i1,i2])
    biggest = max([i1,i2])
    matrice = [[1,2,3],
        [4,5,6],
        [7,8,9],
        ['*',0,'#']]
    if smallest <= 3 and biggest >= 4:
        return matrice[smallest][biggest-4]
    else:
        return "La combinaison linéaire ne permet pas de déterminer une touche"

# Démonstration
On test ici la fiabilité des différentes fonctions crée précédemment

In [231]:
#On crée les vecteurs de notre base orthonomée à partir des fréquences DMTF
u = createOrthonormalBasis(fs,w)

# Si on le souhaite on peut utiliser ce synthétiseur de code DTMF afin de réaliser des tests
# w = codeSynthesizer([0, 0, 0, 10, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0],u)

# On va ici récupérer le code DTMF contenu dans un fichier
sound = getCodeFromFile("files/DTMF/steganographie")

# On récupère les coordonnées de notre son dans la base ainsi que sa projection orthonormée
sound_coeff, sound_product = projectionOrt(u, sound)

# Si on le souhaite on peut utiliser cet appel de fonction afin de vérifier le caractère normé de chaque vecteur de la base
# print(scal(u[1], u[1]))

freq_coeff = findFreqCoeff(sound_coeff)
# On récupère les 2 principales fréquences de la combinaison linéaire correspondant au code DTMF choisis quelques lignes
# auparavant ainsi que leurs indexs
index1, index2, max1, max2 = findingMainFrequencies(freq_coeff)

# On obtiens la touche de clavier liée 2 principales fréquences de la combinaison linéaire
print(whatNumIsIt(index1,index2))

#------ PARTIE SPECIAL STEGANOGRAPHIE ------
# On récupère le message caché
hidden_msg = sound - sound_product

#Ici on écoute le message caché
Audio(hidden_msg, rate=44100)

La combinaison linéaire ne permet pas de déterminer une touche
